In [1]:
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

engine = create_engine('sqlite:///league_2017.sqlite')

Base = automap_base()
Base.prepare(engine, reflect = True)

Base.classes.keys()

['table_2017']

In [2]:
inspector = inspect(engine)

table_columns = inspector.get_columns('table_2017')
for column in table_columns:
    print(column['name']), column['type']

id
Team
Playoffs
Points
Fenwick_Close
SOG_Close
Gpercent_Tied


In [3]:
Table_2017 = Base.classes.table_2017

session = Session(bind = engine)

In [4]:
sel = [
    Table_2017.Team,
    Table_2017.Points,
    Table_2017.Fenwick_Close,
    Table_2017.SOG_Close,
    Table_2017.Gpercent_Tied
]

team_playoff = session.query(*sel).filter(Table_2017.Playoffs == 'Playoff').all()

In [5]:
sel = [
    Table_2017.Team,
    Table_2017.Points,
    Table_2017.Fenwick_Close,
    Table_2017.SOG_Close,
    Table_2017.Gpercent_Tied
]

team_dnq = session.query(*sel).filter(Table_2017.Playoffs == 'DNQ').all()

In [6]:
team_playoff_df = pd.DataFrame(team_playoff)
team_playoff_df.head()

,Team,Points,Fenwick_Close,SOG_Close,Gpercent_Tied
0,ANA,74,54.33,53.81,54.32
1,CHI,79,51.35,51.00,60.00
2,DAL,80,53.54,52.82,47.42
3,DET,69,49.45,48.96,52.04
4,FLA,77,48.90,48.88,61.84


In [7]:
team_playoff_df['Team'].count()

32

In [8]:
team_dnq_df = pd.DataFrame(team_dnq)
team_dnq_df.head()

,Team,Points,Fenwick_Close,SOG_Close,Gpercent_Tied
0,ARI,60,44.81,44.61,50.59
1,BOS,70,50.27,51.62,52.63
2,BUF,55,46.40,47.39,45.26
3,CAR,64,53.01,53.45,46.32
4,CBJ,55,48.74,49.19,42.55


In [9]:
team_dnq_df['Team'].count()

28

### III.05

In [74]:
league_2017_ml = pd.read_csv('csv_data/_training.testing.ml.csv')


league_2017_ml.head()

,Points,Corsi Close,Fenwick Close,SOG Close,G% Down 2,G For Down 2,G Agnst Down 2,G% Down 1,G For Down 1,G Agnst Down 1,...,G For Tied,G Agnst Tied,G% Up 1,G For Up 1,G Agnst Up 1,G% Up 2,G For Up 2,G Agnst Up 2,B For Up 2,Playoffs
0,103,53.68,54.06,53.60,36.36,8,14,45.45,20,24,...,54,52,52.31,34,31,63.64,21,12,71,Playoff
1,78,45.65,45.07,45.33,42.50,17,23,38.46,20,32,...,56,52,43.86,25,32,52.38,11,10,48,DNQ
2,93,49.87,50.06,51.29,34.62,9,17,54.39,31,26,...,63,56,45.33,34,41,55.88,19,15,72,DNQ
3,81,47.02,46.92,48.00,42.42,14,19,43.75,28,36,...,54,65,43.18,19,25,65.52,19,10,45,DNQ
4,86,51.55,50.73,51.30,33.33,10,20,52.24,35,32,...,55,65,41.82,23,32,46.88,15,17,45,DNQ


In [75]:
unique_teams = league_2017_ml['Playoffs'].unique()
len(unique_teams)

2

In [76]:
X = league_2017_ml.drop('Playoffs', axis = 1)
y = league_2017_ml['Playoffs']

In [77]:
X.dtypes

Points              int64
Corsi Close       float64
Fenwick Close     float64
SOG Close         float64
G% Down 2         float64
G For Down 2        int64
G Agnst Down 2      int64
G% Down 1         float64
G For Down 1        int64
G Agnst Down 1      int64
G% Tied           float64
G For Tied          int64
G Agnst Tied        int64
G% Up 1           float64
G For Up 1          int64
G Agnst Up 1        int64
G% Up 2           float64
G For Up 2          int64
G Agnst Up 2        int64
B For Up 2          int64
dtype: object

In [78]:
#data pre-processing

print(X.shape, y.shape)

(240, 20) (240,)


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

In [80]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 1, stratify = y)

In [81]:
X_train.dtypes

Points              int64
Corsi Close       float64
Fenwick Close     float64
SOG Close         float64
G% Down 2         float64
G For Down 2        int64
G Agnst Down 2      int64
G% Down 1         float64
G For Down 1        int64
G Agnst Down 1      int64
G% Tied           float64
G For Tied          int64
G Agnst Tied        int64
G% Up 1           float64
G For Up 1          int64
G Agnst Up 1        int64
G% Up 2           float64
G For Up 2          int64
G Agnst Up 2        int64
B For Up 2          int64
dtype: object

In [82]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [83]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [84]:
encoded_y_train

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0], dtype=int64)

In [85]:
label_encoder.inverse_transform(encoded_y_train)

array(['DNQ', 'DNQ', 'Playoff', 'Playoff', 'Playoff', 'DNQ', 'DNQ', 'DNQ',
       'DNQ', 'Playoff', 'Playoff', 'DNQ', 'DNQ', 'DNQ', 'DNQ', 'Playoff',
       'DNQ', 'DNQ', 'Playoff', 'DNQ', 'Playoff', 'Playoff', 'Playoff',
       'DNQ', 'DNQ', 'DNQ', 'Playoff', 'Playoff', 'Playoff', 'DNQ',
       'Playoff', 'DNQ', 'Playoff', 'Playoff', 'DNQ', 'Playoff', 'Playoff',
       'Playoff', 'DNQ', 'Playoff', 'DNQ', 'Playoff', 'DNQ', 'Playoff',
       'DNQ', 'DNQ', 'Playoff', 'Playoff', 'Playoff', 'Playoff', 'DNQ',
       'DNQ', 'DNQ', 'Playoff', 'Playoff', 'Playoff', 'DNQ', 'DNQ',
       'Playoff', 'Playoff', 'DNQ', 'Playoff', 'Playoff', 'Playoff',
       'Playoff', 'DNQ', 'Playoff', 'DNQ', 'DNQ', 'DNQ', 'Playoff', 'DNQ',
       'DNQ', 'Playoff', 'Playoff', 'Playoff', 'DNQ', 'Playoff', 'DNQ',
       'DNQ', 'Playoff', 'Playoff', 'DNQ', 'Playoff', 'DNQ', 'DNQ', 'DNQ',
       'Playoff', 'DNQ', 'Playoff', 'DNQ', 'Playoff', 'Playoff', 'Playoff',
       'DNQ', 'Playoff', 'DNQ', 'Playoff', 'DNQ', 'Play

In [86]:
#Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [89]:
from keras.models import Sequential
from keras.layers import Dense

#Create model and add layers
model = Sequential()
model.add(Dense(units = 100, activation = 'relu', input_dim = 20))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 2, activation = 'softmax'))

In [90]:
#compile and fit the model
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs = 60,
    shuffle = True,
    verbose = 2)

Epoch 1/60
 - 8s - loss: 0.7157 - acc: 0.4778
Epoch 2/60
 - 0s - loss: 0.5457 - acc: 0.7778
Epoch 3/60
 - 0s - loss: 0.4438 - acc: 0.8222
Epoch 4/60
 - 0s - loss: 0.3727 - acc: 0.8611
Epoch 5/60
 - 0s - loss: 0.3187 - acc: 0.8889
Epoch 6/60
 - 0s - loss: 0.2803 - acc: 0.9000
Epoch 7/60
 - 0s - loss: 0.2504 - acc: 0.9167
Epoch 8/60
 - 0s - loss: 0.2238 - acc: 0.9222
Epoch 9/60
 - 0s - loss: 0.2043 - acc: 0.9389
Epoch 10/60
 - 0s - loss: 0.1860 - acc: 0.9500
Epoch 11/60
 - 0s - loss: 0.1736 - acc: 0.9500
Epoch 12/60
 - 0s - loss: 0.1598 - acc: 0.9500
Epoch 13/60
 - 0s - loss: 0.1494 - acc: 0.9556
Epoch 14/60
 - 0s - loss: 0.1392 - acc: 0.9611
Epoch 15/60
 - 0s - loss: 0.1281 - acc: 0.9556
Epoch 16/60
 - 0s - loss: 0.1196 - acc: 0.9556
Epoch 17/60
 - 0s - loss: 0.1099 - acc: 0.9722
Epoch 18/60
 - 0s - loss: 0.1047 - acc: 0.9667
Epoch 19/60
 - 0s - loss: 0.0980 - acc: 0.9667
Epoch 20/60
 - 0s - loss: 0.0917 - acc: 0.9722
Epoch 21/60
 - 0s - loss: 0.0875 - acc: 0.9722
Epoch 22/60
 - 0s - lo

In [91]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled,
    y_test_categorical,
    verbose = 2)

print(f'Normal Neural Network| Loss: {model_loss}, Accuracy: {model_accuracy}')

Normal Neural Network| Loss: 0.18186618636051813, Accuracy: 0.9333333412806193


In [92]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f'Predicted classes: {prediction_labels}')
print(f'Actual Labels: {list(y_test[:5])}')

Predicted classes: ['DNQ' 'Playoff' 'Playoff' 'Playoff' 'Playoff']
Actual Labels: ['DNQ', 'Playoff', 'Playoff', 'Playoff', 'DNQ']
